In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
import json

# Statsbomb WIP

## Competitions

In [2]:
competitions = pd.read_json('../raw_data/open-data/data/competitions.json')
# adding match count to competitions
for i, row in competitions.iterrows():
    with open(f"../raw_data/open-data/data/matches/{row['competition_id']}/{row['season_id']}.json") as file:
        json_file = json.load(file)
    
    # print(row['competition_id'], row['season_id'], row['competition_name'], row['season_name'], len(json_file))
    competitions.loc[i, 'match_count'] = len(json_file)

In [3]:
feats_to_see = ['competition_id', 'season_id', 'country_name', 'competition_name', 'season_name', 'match_count']
comp_20p_matches = competitions.query('match_count >= 20')[feats_to_see]
comp_20p_matches

,competition_id,season_id,country_name,competition_name,season_name,match_count
0,9,27,Germany,1. Bundesliga,2015/2016,306.0
22,37,90,England,FA Women's Super League,2020/2021,131.0
23,37,42,England,FA Women's Super League,2019/2020,87.0
24,37,4,England,FA Women's Super League,2018/2019,108.0
26,43,106,International,FIFA World Cup,2022,64.0
27,43,3,International,FIFA World Cup,2018,64.0
34,1238,108,India,Indian Super league,2021/2022,115.0
35,11,90,Spain,La Liga,2020/2021,35.0
36,11,42,Spain,La Liga,2019/2020,33.0
37,11,4,Spain,La Liga,2018/2019,34.0


In [4]:
competitions_id = comp_20p_matches.query('match_count > 38')[['competition_id', 'season_id']].reset_index(drop=True)
competitions_id.loc[len(competitions_id)] = [53, 106]
competitions_id

,competition_id,season_id
0,9,27
1,37,90
2,37,42
3,37,4
4,43,106
5,43,3
6,1238,108
7,11,27
8,7,27
9,2,27


## Matches

In [5]:
def get_clean_match(df):
    '''
    This function receives a match dataframe and returns it without uninteresting
    columns.
    '''
    cols_to_drop = ['match_status', 'match_status_360', 'last_updated', 'last_updated_360', 'metadata', 'stadium', 'referee']
    df = df.drop(columns=cols_to_drop)
    return df

In [6]:
match1 = pd.read_json('../raw_data/open-data/data/matches/2/27.json')
match1_clean = get_clean_match(match1)
match1_clean

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage
0,3754058,2016-01-02,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 22, 'home_team_name': 'Leices...","{'away_team_id': 28, 'away_team_name': 'AFC Bo...",0,0,20,"{'id': 1, 'name': 'Regular Season'}"
1,3754245,2015-10-17,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 27, 'home_team_name': 'West B...","{'away_team_id': 41, 'away_team_name': 'Sunder...",1,0,9,"{'id': 1, 'name': 'Regular Season'}"
2,3754136,2015-12-19,18:30:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 37, 'home_team_name': 'Newcas...","{'away_team_id': 59, 'away_team_name': 'Aston ...",1,1,17,"{'id': 1, 'name': 'Regular Season'}"
3,3754037,2016-04-30,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 29, 'home_team_name': 'Everto...","{'away_team_id': 28, 'away_team_name': 'AFC Bo...",2,1,36,"{'id': 1, 'name': 'Regular Season'}"
4,3754039,2016-02-13,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 31, 'home_team_name': 'Crysta...","{'away_team_id': 23, 'away_team_name': 'Watfor...",1,2,26,"{'id': 1, 'name': 'Regular Season'}"
...,...,...,...,...,...,...,...,...,...,...,...
375,3754020,2015-08-17,21:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 24, 'home_team_name': 'Liverp...","{'away_team_id': 28, 'away_team_name': 'AFC Bo...",1,0,2,"{'id': 1, 'name': 'Regular Season'}"
376,3754267,2015-08-15,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 23, 'home_team_name': 'Watfor...","{'away_team_id': 27, 'away_team_name': 'West B...",0,0,2,"{'id': 1, 'name': 'Regular Season'}"
377,3754141,2015-08-09,14:30:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 1, 'home_team_name': 'Arsenal...","{'away_team_id': 40, 'away_team_name': 'West H...",0,2,1,"{'id': 1, 'name': 'Regular Season'}"
378,3754128,2015-08-08,16:00:00.000,"{'competition_id': 2, 'country_name': 'England...","{'season_id': 27, 'season_name': '2015/2016'}","{'home_team_id': 28, 'home_team_name': 'AFC Bo...","{'away_team_id': 59, 'away_team_name': 'Aston ...",0,1,1,"{'id': 1, 'name': 'Regular Season'}"


## Events

In [4]:
evs1 = pd.read_json('../raw_data/open-data/data/events/15978.json')
evs1.head(1)

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,off_camera,player,position,location,pass,carry,under_pressure,interception,dribble,counterpress,duel,ball_receipt,ball_recovery,out,clearance,shot,goalkeeper,foul_won,foul_committed,substitution,block,bad_behaviour
0,c49b57c0-8b25-4183-acd7-b13a52464607,1,1,2023-09-05,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 210, 'name': 'Real Sociedad'}",0.0,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# to add id columns
for feat in ['type', 'team']:
  evs1.loc[:, f'{feat}_id'] = evs1.loc[:, feat].map(lambda x: x['id'])
  evs1.loc[:, f'{feat}_name'] = evs1.loc[:, feat].map(lambda x: x['name'])
  
evs1.head(3)

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,off_camera,player,position,location,pass,carry,under_pressure,interception,dribble,counterpress,duel,ball_receipt,ball_recovery,out,clearance,shot,goalkeeper,foul_won,foul_committed,substitution,block,bad_behaviour,type_id,type_name,team_id,team_name
0,c49b57c0-8b25-4183-acd7-b13a52464607,1,1,2023-09-05,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 210, 'name': 'Real Sociedad'}",0.0,"{'formation': 442, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,Starting XI,210,Real Sociedad
1,129dbbf1-1faf-48cd-a550-022a5154cb90,2,1,2023-09-05,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",0.0,"{'formation': 433, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,Starting XI,217,Barcelona
2,19a741d4-8122-41b2-8ea6-02769da5ff5d,3,1,2023-09-05,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 210, 'name': 'Real Sociedad'}",0.0,NaN,[957a518f-23a0-43c7-9b16-fce3911d3e3f],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,Half Start,210,Real Sociedad


In [28]:
evs1.groupby(by=['type_id', 'type_name', 'team_id']).size()

type_id  type_name       team_id
2        Ball Recovery   210         34
                         217         70
3        Dispossessed    210         14
                         217          8
4        Duel            210         38
                         217         27
6        Block           210         21
                         217         13
9        Clearance       210         21
                         217          4
10       Interception    210         21
                         217         10
14       Dribble         210          6
                         217         26
16       Shot            210          8
                         217         12
17       Pressure        210        141
                         217        112
18       Half Start      210          2
                         217          2
19       Substitution    210          3
                         217          3
21       Foul Won        210          8
                         217         19
22     

In [79]:
# pass analysis
pass_outcome = evs1.query('type_id == 30')['pass'].map(lambda x: x.get('outcome'))
pass_outcome.isna().sum()/len(pass_outcome)
# pass_outcome[~pass_outcome.isna()].map(lambda x: x.get('name'))

0.8336236933797909

In [70]:
pass_recipient = evs1.query('type_id == 30')['pass'].map(lambda x: x.get('recipient'))
pass_recipient

4       {'id': 5573, 'name': 'Héctor Alfredo Moreno He...
6                                                    None
9            {'id': 6374, 'name': 'Nélson Cabral Semedo'}
12          {'id': 5213, 'name': 'Gerard Piqué Bernabéu'}
15             {'id': 5492, 'name': 'Samuel Yves Umtiti'}
                              ...                        
3884         {'id': 6676, 'name': 'Igor Zubeldia Elorza'}
3886     {'id': 6605, 'name': 'Joseba Zaldúa Bengoetxea'}
3891           {'id': 3042, 'name': 'Mikel Merino Zazón'}
3899                                                 None
3901     {'id': 5203, 'name': 'Sergio Busquets i Burgos'}
Name: pass, Length: 1148, dtype: object

In [69]:
# seems like recipient exists iff pass is completed
print(len(pass_recipient))
pass_recipient.isna() == ~pass_outcome.isna()

1148


4       True
6       True
9       True
12      True
15      True
        ... 
3884    True
3886    True
3891    True
3899    True
3901    True
Name: pass, Length: 1148, dtype: bool

In [74]:
evs1.loc[:, 'pass']#.map(lambda x: x.get('recipient'))

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4       {'recipient': {'id': 5573, 'name': 'Héctor Alf...
                              ...                        
3900                                                  NaN
3901    {'recipient': {'id': 5203, 'name': 'Sergio Bus...
3902                                                  NaN
3903                                                  NaN
3904                                                  NaN
Name: pass, Length: 3905, dtype: object

In [77]:
passes = evs1.query('type_id == 30')
passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 1 if x.get('recipient') else 0)
passes

/tmp/ipykernel_2699/2401062557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes.loc[:, 'pass_completed'] = passes.loc[:, 'pass'].map(lambda x: 1 if x.get('recipient') else 0)


,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,off_camera,player,position,location,pass,carry,under_pressure,interception,dribble,counterpress,duel,ball_receipt,ball_recovery,out,clearance,shot,goalkeeper,foul_won,foul_committed,substitution,block,bad_behaviour,type_id,type_name,team_id,team_name,pass_completed
4,d652dea1-a7c5-4140-9ae8-f893acd0b72d,5,1,2023-09-05 00:00:00.120,0,0,"{'id': 30, 'name': 'Pass'}",2,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 9, 'name': 'From Kick Off'}","{'id': 210, 'name': 'Real Sociedad'}",3.239211,NaN,[1d614951-0549-465a-9a4c-106287f86e64],1.0,"{'id': 6685, 'name': 'Mikel Oyarzabal Ugarte'}","{'id': 12, 'name': 'Right Midfield'}","[60.0, 40.0]","{'recipient': {'id': 5573, 'name': 'Héctor Alf...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,210,Real Sociedad,1
6,13e796c1-812a-4c25-a910-05982c1652df,7,1,2023-09-05 00:00:05.397,0,5,"{'id': 30, 'name': 'Pass'}",2,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 9, 'name': 'From Kick Off'}","{'id': 210, 'name': 'Real Sociedad'}",3.740729,NaN,[46187568-527b-4502-be1d-8985a1d4c7c7],NaN,"{'id': 5573, 'name': 'Héctor Alfredo Moreno He...","{'id': 5, 'name': 'Left Center Back'}","[49.9, 39.3]","{'length': 14.723111, 'angle': -1.997986, 'hei...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,210,Real Sociedad,0
9,0fbebb10-09b0-454a-a1e3-79fdeb0a2038,10,1,2023-09-05 00:00:10.315,0,10,"{'id': 30, 'name': 'Pass'}",3,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",1.529271,NaN,[bdca08b8-080b-4623-9b70-842566126ea3],NaN,"{'id': 5492, 'name': 'Samuel Yves Umtiti'}","{'id': 5, 'name': 'Left Center Back'}","[27.3, 23.9]","{'recipient': {'id': 6374, 'name': 'Nélson Cab...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,217,Barcelona,1
12,f99f883b-4973-47f0-a227-0774c4b79545,13,1,2023-09-05 00:00:16.237,0,16,"{'id': 30, 'name': 'Pass'}",3,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",1.871526,NaN,[6a0f55e5-f8bb-4e94-a549-a6f7052a5297],NaN,"{'id': 6374, 'name': 'Nélson Cabral Semedo'}","{'id': 2, 'name': 'Right Back'}","[26.1, 76.4]","{'recipient': {'id': 5213, 'name': 'Gerard Piq...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,217,Barcelona,1
15,d615902f-c8d6-44d1-9819-aaf92d3f3b19,16,1,2023-09-05 00:00:19.368,0,19,"{'id': 30, 'name': 'Pass'}",3,"{'id': 217, 'name': 'Barcelona'}","{'id': 1, 'name': 'Regular Play'}","{'id': 217, 'name': 'Barcelona'}",2.012964,NaN,[fc6e5557-120a-46ae-8973-be43c1bbbc4a],NaN,"{'id': 5213, 'name': 'Gerard Piqué Bernabéu'}","{'id': 3, 'name': 'Right Center Back'}","[13.6, 54.6]","{'recipient': {'id': 5492, 'name': 'Samuel Yve...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,217,Barcelona,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3884,a775043d-bc6d-448b-9412-3b33b3383658,3885,2,2023-09-05 00:47:12.375,92,12,"{'id': 30, 'name': 'Pass'}",175,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 4, 'name': 'From Throw In'}","{'id': 210, 'name': 'Real Sociedad'}",1.554603,NaN,[0f5253c4-a32c-461e-bb5b-d4f664b30e8a],1.0,"{'id': 6605, 'name': 'Joseba Zaldúa Bengoetxea'}","{'id': 2, 'name': 'Right Back'}","[89.1, 80.0]","{'recipient': {'id': 6676, 'name': 'Igor Zubel...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,Pass,210,Real Sociedad,1
3886,28761d2b-bc7e-4dac-9e11-8d6ae6938183,3887,2,2023-09-05 00:47:14.010,92,14,"{'id': 30, 'name': 'Pass'}",175,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 4, 'name': 'From Throw In'}","{'id': 210, 'name': 'Real Sociedad'}",2.426845,NaN,[11ef2120-7a31-41c7-b882-3e75229b4dd1],NaN,"{'id': 6676, 'name': 'Igor Zubeldia Elorza'}

In [85]:
# shot analysis
evs1.query('type_id == 16')['shot'].iloc[0]

{'statsbomb_xg': 0.05368826,
 'end_location': [120.0, 36.5, 1.1],
 'outcome': {'id': 97, 'name': 'Goal'},
 'first_time': True,
 'technique': {'id': 91, 'name': 'Half Volley'},
 'body_part': {'id': 38, 'name': 'Left Foot'},
 'type': {'id': 87, 'name': 'Open Play'},
 'freeze_frame': [{'location': [114.8, 29.3],
   'player': {'id': 5477, 'name': 'Ousmane Dembélé'},
   'position': {'id': 21, 'name': 'Left Wing'},
   'teammate': False},
  {'location': [109.6, 36.0],
   'player': {'id': 5470, 'name': 'Ivan Rakitić'},
   'position': {'id': 10, 'name': 'Center Defensive Midfield'},
   'teammate': False},
  {'location': [113.0, 26.9],
   'player': {'id': 6379, 'name': 'Sergi Roberto Carnicer'},
   'position': {'id': 13, 'name': 'Right Center Midfield'},
   'teammate': False},
  {'location': [109.7, 31.3],
   'player': {'id': 5213, 'name': 'Gerard Piqué Bernabéu'},
   'position': {'id': 3, 'name': 'Right Center Back'},
   'teammate': False},
  {'location': [103.7, 44.7],
   'player': {'id': 5211

In [93]:
shot_on_target_ids = [96, 97, 100]
goal_id = 97
shots = evs1.query('type_id == 16')
shots.loc[:, 'shot_on_target'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') in shot_on_target_ids else 0)
shots.loc[:, 'shot_goal'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') == goal_id else 0)

shots

/tmp/ipykernel_2699/2105986794.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots.loc[:, 'shot_on_target'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') in shot_on_target_ids else 0)
/tmp/ipykernel_2699/2105986794.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shots.loc[:, 'shot_goal'] = shots.loc[:, 'shot'].map(lambda x: 1 if x.get('outcome').get('id') == goal_id else 0)


,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,off_camera,player,position,location,pass,carry,under_pressure,interception,dribble,counterpress,duel,ball_receipt,ball_recovery,out,clearance,shot,goalkeeper,foul_won,foul_committed,substitution,block,bad_behaviour,type_id,type_name,team_id,team_name,shot_on_target,shot_goal
586,73b4f96a-2a64-4f7c-8fb1-0729c921e99f,587,1,2023-09-05 00:11:54.021,11,54,"{'id': 16, 'name': 'Shot'}",23,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 3, 'name': 'From Free Kick'}","{'id': 210, 'name': 'Real Sociedad'}",0.750514,NaN,[3ff9335f-c213-4f6b-a824-34a247f97df2],NaN,"{'id': 6679, 'name': 'Aritz Elustondo Irribaria'}","{'id': 3, 'name': 'Right Center Back'}","[107.5, 29.7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.05368826, 'end_location': [...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,210,Real Sociedad,1,1
728,176d86cf-3674-4f8c-9c06-9b199a10d0a9,729,1,2023-09-05 00:16:26.099,16,26,"{'id': 16, 'name': 'Shot'}",32,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 1, 'name': 'Regular Play'}","{'id': 210, 'name': 'Real Sociedad'}",0.753255,NaN,[46dceb1f-5dcb-4493-9e0d-240bdb0af5c1],NaN,"{'id': 6695, 'name': 'Juan Miguel Jiménez López'}","{'id': 24, 'name': 'Left Center Forward'}","[99.4, 50.9]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.048043124, 'end_location': ...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,210,Real Sociedad,0,0
1595,66da79ef-f087-4d40-805a-b07a280f05aa,1596,1,2023-09-05 00:35:05.307,35,5,"{'id': 16, 'name': 'Shot'}",63,"{'id': 217, 'name': 'Barcelona'}","{'id': 2, 'name': 'From Corner'}","{'id': 217, 'name': 'Barcelona'}",1.232881,NaN,"[386845a5-f11b-4e70-a826-e73f8065cd90, a7273ca...",NaN,"{'id': 5213, 'name': 'Gerard Piqué Bernabéu'}","{'id': 3, 'name': 'Right Center Back'}","[112.8, 46.7]",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.059001160000000004, 'end_lo...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,217,Barcelona,0,0
1810,c9deaf75-3381-49fd-beb2-6e078d0971f5,1811,1,2023-09-05 00:40:51.538,40,51,"{'id': 16, 'name': 'Shot'}",73,"{'id': 217, 'name': 'Barcelona'}","{'id': 2, 'name': 'From Corner'}","{'id': 217, 'name': 'Barcelona'}",0.271398,NaN,[1ac15525-ccbe-4d9b-ac27-68fe4e6f3d04],NaN,"{'id': 5213, 'name': 'Gerard Piqué Bernabéu'}","{'id': 3, 'name': 'Right Center Back'}","[113.1, 35.4]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.14770305, 'end_location': [...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,217,Barcelona,1,0
1815,7a635e80-a845-4933-93a1-f810d9c749d6,1816,1,2023-09-05 00:41:57.824,41,57,"{'id': 16, 'name': 'Shot'}",74,"{'id': 217, 'name': 'Barcelona'}","{'id': 2, 'name': 'From Corner'}","{'id': 217, 'name': 'Barcelona'}",1.364274,NaN,"[9e1016b7-11b2-4b0d-8122-b0224009d985, d7753c8...",NaN,"{'id': 5470, 'name': 'Ivan Rakitić'}","{'id': 10, 'name': 'Center Defensive Midfield'}","[112.4, 38.0]",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.100465104, 'end_location': ...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,217,Barcelona,0,0
2319,cfceefce-f6b8-49e6-ae55-d3b4fa6950d2,2320,2,2023-09-05 00:08:02.953,53,2,"{'id': 16, 'name': 'Shot'}",97,"{'id': 210, 'name': 'Real Sociedad'}","{'id': 2, 'name': 'From Corner'}","{'id': 210, 'name': 'Real Sociedad'}",1.317323,NaN,[eb45b32c-68f5-4a27-988f-20dda81cf199],NaN,"{'id': 6685, 'name': 'Mikel Oyarzabal Ugarte'}","{'id': 12, 'name': 'Right Midfield'}","[105.4, 34.7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.05929953600000001, 'end_loc...",NaN,NaN,NaN,NaN,NaN,NaN,16,Shot,210,Real Sociedad,1,0
2526,56984762-35f3-4831-ba6a-d66fbe73a59f,2527,2,2023-09-05 00:12:00.585,57,0,"{'id': 16, 'name': 'Shot'}",102,"{'id': 217, 'name': 'Barcelona'}","{'id': 4, 'name': 'From Throw In'}","{'id': 217, 'name': 'Barcelona'}",2.374076,NaN,[94807271-baea-4cab-97b3-def060c66bc5],NaN,"{'id': 5477, 'name': 'Ousmane Dembélé'}","{'id': 21, 'name': 'Left Wing'}","[111.0, 31.5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,